In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(
    'earthquakes.csv', 
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

In [3]:
# New columns get added to the right of the dataframe and can be a single value:
df['noes'] = 1
df.head()

,mag,place,time,alert,tsunami,magType,title,noes
0,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1731369384328,NaN,0,mb,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R...",1
1,0.68,"7 km WNW of Cobb, CA",1731367157080,NaN,0,md,"M 0.7 - 7 km WNW of Cobb, CA",1
2,1.90,"20 km W of Point MacKenzie, Alaska",1731366482588,NaN,0,ml,"M 1.9 - 20 km W of Point MacKenzie, Alaska",1
3,2.15,"4 km W of Magalia, CA",1731366324690,NaN,0,md,"M 2.2 - 4 km W of Magalia, CA",1
4,2.00,"33 km WNW of Petersville, Alaska",1731365892605,NaN,0,ml,"M 2.0 - 33 km WNW of Petersville, Alaska",1


In [4]:
# or a Boolean mask:
df['mah_negative'] = df.mag < 0
df.head()

,mag,place,time,alert,tsunami,magType,title,noes,mah_negative
0,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1731369384328,NaN,0,mb,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R...",1,False
1,0.68,"7 km WNW of Cobb, CA",1731367157080,NaN,0,md,"M 0.7 - 7 km WNW of Cobb, CA",1,False
2,1.90,"20 km W of Point MacKenzie, Alaska",1731366482588,NaN,0,ml,"M 1.9 - 20 km W of Point MacKenzie, Alaska",1,False
3,2.15,"4 km W of Magalia, CA",1731366324690,NaN,0,md,"M 2.2 - 4 km W of Magalia, CA",1,False
4,2.00,"33 km WNW of Petersville, Alaska",1731365892605,NaN,0,ml,"M 2.0 - 33 km WNW of Petersville, Alaska",1,False


In [5]:
# Adding the parsed_place column:
# We have an entity recognition problem on our hands with the place column. There are several entities that have multiple names in the data
#(e.g., CA and California, NV and Nevada):
df.place.str.extract(r', (.*$)')[0].sort_values().unique()

array(['Afghanistan', 'Alaska', 'Aleutian Islands, Alaska', 'Argentina',
       'Arizona', 'Arkansas', 'B.C., MX', 'Bolivia', 'CA', 'California',
       'Canada', 'Chile', 'China', 'Colombia', 'Colorado', 'Costa Rica',
       'Cuba', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Ethiopia', 'Fiji', 'French Southern Territories', 'Georgia',
       'Greece', 'Guadeloupe', 'Guatemala', 'Hawaii', 'Idaho', 'Illinois',
       'India', 'Indonesia', 'Iran', 'Italy', 'Japan', 'Japan region',
       'Kyrgyzstan', 'Maine', 'Massachusetts', 'Mexico', 'Micronesia',
       'Missouri', 'Mongolia', 'Montana', 'Myanmar', 'Nevada',
       'New Jersey', 'New Mexico', 'New Zealand', 'Nicaragua',
       'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon',
       'Pakistan', 'Panama', 'Papua New Guinea', 'Peru', 'Philippines',
       'Puerto Rico', 'Russia', 'Solomon Islands', 'Taiwan', 'Tajikistan',
       'Tennessee', 'Texas', 'Timor Leste', 'Tonga', 'Turkey',
       'U.S. Virgin Is

In [6]:
# Replace parts of the place names to fit our needs:
df['parsed_place'] = df.place.str.replace(
    r'.* of ', '' # remove anything saying <something> of <something>
).str.replace(
    r'the ', '' # remove things starting with "the"
).str.replace(
    r'CA$', 'California' # fix California
).str.replace(
    r'NV$', 'Nevada' # fix Nevada
).str.replace(
    r'MX$', 'Mexico' # fix Mexico
).str.replace(
    r' region$', '' # chop off endings with "region"
).str.replace(
    r'northern ', '' # remove "northern"
).str.replace(
    r'Fiji Islands', 'Fiji' # line up the Fiji places
).str.replace(
    r'^.*, ', '' # remove anything else extraneous from the beginning
).str.strip() # remove any extra spaces

In [7]:
# Now we can use a single name to get all earthquakes for that place:
df.parsed_place.sort_values().unique()

array(['0 km  of The Geysers, CA', '0 km E of Pāhala, Hawaii',
       '0 km ENE of Sun Valley, Nevada', ...,
       'southern Mid-Atlantic Ridge', 'western Indian-Antarctic Ridge',
       'western Xizang'], dtype=object)

In [8]:
# Using the assign() method to create columns
# To create many columns at once or update existing columns, we can use assign():
df.assign(in_ca=df.parsed_place.str.endswith('California'), in_alaska=df.parsed_place.str.endswith('Alaska')).head()

,mag,place,time,alert,tsunami,magType,title,noes,mah_negative,parsed_place,in_ca,in_alaska
0,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1731369384328,NaN,0,mb,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R...",1,False,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",False,False
1,0.68,"7 km WNW of Cobb, CA",1731367157080,NaN,0,md,"M 0.7 - 7 km WNW of Cobb, CA",1,False,"7 km WNW of Cobb, CA",False,False
2,1.90,"20 km W of Point MacKenzie, Alaska",1731366482588,NaN,0,ml,"M 1.9 - 20 km W of Point MacKenzie, Alaska",1,False,"20 km W of Point MacKenzie, Alaska",False,True
3,2.15,"4 km W of Magalia, CA",1731366324690,NaN,0,md,"M 2.2 - 4 km W of Magalia, CA",1,False,"4 km W of Magalia, CA",False,False
4,2.00,"33 km WNW of Petersville, Alaska",1731365892605,NaN,0,ml,"M 2.0 - 33 km WNW of Petersville, Alaska",1,False,"33 km WNW of Petersville, Alaska",False,True


In [9]:
# Concatenation
# Say we were working with two separate dataframes, one with earthquakes accompanied by tsunamis and the other with earthquakes without tsunamis.
# If we wanted to look at earthquakes as a whole, we would want to concatenate the dataframes into a single one:
# we would want to concatenate the dataframes into a single one:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]

tsunami.shape, no_tsunami.shape

((4, 10), (6742, 10))

In [10]:
pd.concat([tsunami, no_tsunami]).shape

(6746, 10)

In [11]:
# We have been working with a subset of the columns from the CSV file, but now we want to get some of the columns we ignored when we read in the data.
# Since we have added new columns in this notebook, we won't want to read in the file again and perform those operations again. Instead, 
# we will concatenate along the columns (axis=1) to add back what we are missing:

additional_columns = pd.read_csv('earthquakes.csv', usecols=['tz', 'felt', 'ids'])
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,mag,place,time,alert,tsunami,magType,title,noes,mah_negative,parsed_place,tz,felt,ids
0,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1731369384328,NaN,0,mb,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R...",1,False,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",NaN,NaN,",usauto7000nra5,us7000nra5,"
1,0.68,"7 km WNW of Cobb, CA",1731367157080,NaN,0,md,"M 0.7 - 7 km WNW of Cobb, CA",1,False,"7 km WNW of Cobb, CA",NaN,NaN,",nc75085486,"


In [12]:
# Notice what happens if the index doesn't align though:

additional_columns = pd.read_csv(
    'earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat(
    [df.head(2), additional_columns.head(2)], axis=1
)

,mag,place,time,alert,tsunami,magType,title,noes,mah_negative,parsed_place,tz,felt,ids
0,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1.731369e+12,NaN,0.0,mb,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R...",1.0,False,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",NaN,NaN,NaN
1,0.68,"7 km WNW of Cobb, CA",1.731367e+12,NaN,0.0,md,"M 0.7 - 7 km WNW of Cobb, CA",1.0,False,"7 km WNW of Cobb, CA",NaN,NaN,NaN
1731369384328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",usauto7000nra5,us7000nra5,"
1731367157080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",nc75085486,"


In [13]:
# If the index doesn't align, we can align it before attempting the concatentation, which we will discuss in chapter 3:
# Say we want to join the tsunami and no_tsunami dataframes, but the no_tsunami dataframe has an additional column. The join parameter
# specifies how to handle any overlap in column names (when appending to the bottom) or in row names (when concatenating to the left/right).
# By default, this is outer, so we keep everything, however, if we use inner, we will only keep what is in common:

pd.concat(
    [tsunami.head(2),
     no_tsunami.head(2).assign(type='earthquak')
    ], join='inner'
)

,mag,place,time,alert,tsunami,magType,title,noes,mah_negative,parsed_place
228,6.80,"40 km SSW of Bartolomé Masó, Cuba",1731257390382,yellow,1,mww,"M 6.8 - 40 km SSW of Bartolomé Masó, Cuba",1,False,"40 km SSW of Bartolomé Masó, Cuba"
1812,5.80,"81 km ESE of Adak, Alaska",1730631408534,green,1,mww,"M 5.8 - 81 km ESE of Adak, Alaska",1,False,"81 km ESE of Adak, Alaska"
0,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1731369384328,NaN,0,mb,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R...",1,False,"191 km ESE of Ust’-Kamchatsk Staryy, Russia"
1,0.68,"7 km WNW of Cobb, CA",1731367157080,NaN,0,md,"M 0.7 - 7 km WNW of Cobb, CA",1,False,"7 km WNW of Cobb, CA"


In [14]:
# In addition, we use ignore_index, since the index doesn't mean anything for us here. 
# This gives us sequential values instead of what we had in the previous result:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

,mag,place,time,alert,tsunami,magType,title,noes,mah_negative,parsed_place
0,6.80,"40 km SSW of Bartolomé Masó, Cuba",1731257390382,yellow,1,mww,"M 6.8 - 40 km SSW of Bartolomé Masó, Cuba",1,False,"40 km SSW of Bartolomé Masó, Cuba"
1,5.80,"81 km ESE of Adak, Alaska",1730631408534,green,1,mww,"M 5.8 - 81 km ESE of Adak, Alaska",1,False,"81 km ESE of Adak, Alaska"
2,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1731369384328,NaN,0,mb,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R...",1,False,"191 km ESE of Ust’-Kamchatsk Staryy, Russia"
3,0.68,"7 km WNW of Cobb, CA",1731367157080,NaN,0,md,"M 0.7 - 7 km WNW of Cobb, CA",1,False,"7 km WNW of Cobb, CA"


In [15]:
# Columns can be deleted using dictionary syntax with del:
del df['noes']
df.columns

Index(['mag', 'place', 'time', 'alert', 'tsunami', 'magType', 'title',
       'mah_negative', 'parsed_place'],
      dtype='object')

In [16]:
# If we don't know if the column exists, we can use a try/except block:
try:
    del df['noes']
except KeyError:
    # handle the error here
    print('not there anymore')

not there anymore


In [17]:
# We can also use pop(). This will allow us to use the series we remove later. Note there will be an error if the key doesn't exist, 
# so we can also use a try/except here:

mah_negative = df.pop('mah_negative')
df.columns

Index(['mag', 'place', 'time', 'alert', 'tsunami', 'magType', 'title',
       'parsed_place'],
      dtype='object')

In [18]:
mah_negative.value_counts()

mah_negative
False    6192
True      554
Name: count, dtype: int64

In [19]:
df[mah_negative].head()

,mag,place,time,alert,tsunami,magType,title,parsed_place
66,-0.28,"3 km NE of Hebgen Lake Estates, Montana",1731335013560,NaN,0,md,"M -0.3 - 3 km NE of Hebgen Lake Estates, Montana","3 km NE of Hebgen Lake Estates, Montana"
87,-0.05,"16 km ESE of Little Lake, CA",1731324657980,NaN,0,ml,"M -0.1 - 16 km ESE of Little Lake, CA","16 km ESE of Little Lake, CA"
116,-0.20,"45 km WNW of Indian Springs, Nevada",1731313548721,NaN,0,ml,"M -0.2 - 45 km WNW of Indian Springs, Nevada","45 km WNW of Indian Springs, Nevada"
185,-0.15,"66 km WNW of Beluga, Alaska",1731278262530,NaN,0,ml,"M -0.2 - 66 km WNW of Beluga, Alaska","66 km WNW of Beluga, Alaska"
203,-0.10,"47 km ESE of Beatty, Nevada",1731271742957,NaN,0,ml,"M -0.1 - 47 km ESE of Beatty, Nevada","47 km ESE of Beatty, Nevada"


In [20]:
# We can drop rows by passing a list of indices to the drop() method. Notice in the following example that when asking for the first 2 rows with head() 
# we get the 3rd and 4th rows because we dropped the original first 2 with drop([0, 1]):

df.drop([0, 1]).head(2)

,mag,place,time,alert,tsunami,magType,title,parsed_place
2,1.90,"20 km W of Point MacKenzie, Alaska",1731366482588,NaN,0,ml,"M 1.9 - 20 km W of Point MacKenzie, Alaska","20 km W of Point MacKenzie, Alaska"
3,2.15,"4 km W of Magalia, CA",1731366324690,NaN,0,md,"M 2.2 - 4 km W of Magalia, CA","4 km W of Magalia, CA"


In [21]:
# The drop() method drops along the row axis by default. If we pass in axis=1 for the column axis, we can delete columns:
df.drop(columns=[col for col in df.columns \
                 if col not in \
                 ['alert', 'mag', 'title', 'tsunami']]
                 ).head()

,mag,alert,tsunami,title
0,4.90,NaN,0,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R..."
1,0.68,NaN,0,"M 0.7 - 7 km WNW of Cobb, CA"
2,1.90,NaN,0,"M 1.9 - 20 km W of Point MacKenzie, Alaska"
3,2.15,NaN,0,"M 2.2 - 4 km W of Magalia, CA"
4,2.00,NaN,0,"M 2.0 - 33 km WNW of Petersville, Alaska"


In [22]:
# We also have the option of passing the column names to the columns parameter instead of using axis=1:
df.drop(
    columns=[col for col in df.columns \
             if col not in \
             ['alert', 'mag', 'title', 'time', 'tsunami']]
).equals(
    df.drop(
        [col for col in df.columns \
         if col not in ['alert', 'mag', 'title', 'time', 'tsunami']],
        axis=1
    )
)

True

In [23]:
# By default, drop(), along with the majority of DataFrame methods, will return a new DataFrame object. If we just want to change the one we are working with, we can pass inplace=True. 
# This should be used with care:
df.drop(
    columns=[col for col in df.columns \
             if col not in \
             ['alert', 'mag', 'parsed_place', 'time', 'tsunami']],
    inplace=True
)
df.head()


,mag,time,alert,tsunami,parsed_place
0,4.90,1731369384328,NaN,0,"191 km ESE of Ust’-Kamchatsk Staryy, Russia"
1,0.68,1731367157080,NaN,0,"7 km WNW of Cobb, CA"
2,1.90,1731366482588,NaN,0,"20 km W of Point MacKenzie, Alaska"
3,2.15,1731366324690,NaN,0,"4 km W of Magalia, CA"
4,2.00,1731365892605,NaN,0,"33 km WNW of Petersville, Alaska"
